In [1]:
import vonkarmanFT as vk
import vK2KGPR
import GPRutils

import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import scipy.optimize as opt
from scipy.spatial.ckdtree import cKDTree

In [2]:
"""Code used to extract, calculate the GP, and then save it in an NPZ"""
# dataC = GPRutils.dataContainer("folio2", 465)
# dataC.loadFITS()
# dataC.extractData()
# dataC.sigmaClip()
# dataC.splitData()
# GP = vK2KGPR.vonKarman2KernelGPR(dataC, printing=True)

# # Really good parameters for exposure 465 from FoM A
# params = np.array([
#     206.400264,
#     17.376640,
#     -0.000061,
#     0.017663,
#     0.036065,
#     1
# ])

# GP.fit(params)
# GP.predict(dataC.Xtest)

# dataC.saveNPZ("forProf")

'Code used to extract, calculate the GP, and then save it in an NPZ'

In [3]:
"""Load data from npz file"""
pathToData = "465.npz"
dataC = GPRutils.loadNPZ(pathToData)

In [14]:
"""Here are all of the attributes of the dataContainer object:"""
# exposure number from austinFull.fits
dataC.expNum

# (N, 2) array of star positions (u, v)
dataC.X, dataC.Xtrain, dataC.Xtest

# (N, 2) array of residuals (dx, dy)
dataC.Y, dataC.Ytrain, dataC.Ytest

# (N, 2) array of shot noise error. Both columns are identical
dataC.E, dataC.Etrain, dataC.Etest

# fraction of data that is used for training
dataC.train_size

# fraction of data used for creating plots and calculating figure of merit
dataC.test_size

# total number of stars used
dataC.nData

# total number of stars in the training set
dataC.nTrain

# total number of stars in the test set
dataC.nTest

# (5,) array of parameters: variance, outerScale, diameter, wind_x, wind_y, 
# and white_noise_variance.
dataC.params

# (nTest, 2) array of posterior predictive mean of residuals
dataC.fbar_s

array([[-10.93171023,   6.98464864],
       [  4.67918777,   3.45967709],
       [ 19.15431965,  -5.29753775],
       ...,
       [ 19.36033955,   0.11071888],
       [ 12.84685666,  -4.93530497],
       [ 12.17754574,   5.15527327]])

In [16]:
"""
This is how the fitting is done in vonKarman2KernelGPR.fit from vK2KGPR.py.

Note that GPRutils.flat is a function that simply flattens (N, 2) arrays into 
(2N,) arrays because this is the shape needed to do this linear algebra.
"""

ttt = vk.TurbulentLayer(
    variance=dataC.params[0],
    outerScale=dataC.params[1],
    diameter=dataC.params[2],
    wind=(dataC.params[3], dataC.params[4]))

du, dv = GPRutils.getGrid(dataC.Xtrain, dataC.Xtrain)
Cuv = ttt.getCuv(du, dv)
Cuv[:, :, 0, 1] *= 0 # set Cuv to 0
Cuv[:, :, 1, 0] *= 0 # set Cvu to 0
n1, n2 = Cuv.shape[0], Cuv.shape[1]

K = np.swapaxes(Cuv, 1, 2).reshape(2*n1, 2*n2)
W = np.diag(GPRutils.flat(dataC.Etrain)**2) * dataC.params[5]
L = np.linalg.cholesky(K + W)

alpha = np.linalg.solve(L, GPRutils.flat(dataC.Ytrain))
alpha = np.linalg.solve(L.T, alpha)

In [17]:
"""
Now that the model is trained on a particular set of parameters, we can use 
this code to predict residuals for a set of new star positions, X. In this 
case, we use the testing set.

Note that GPRutils.unflat just makes a (2N,) array into a (N, 2) array since (N, 2) is a more sensible shape when plotting.
"""
X = dataC.Xtest

du, dv = GPRutils.getGrid(X, dataC.Xtrain)
Cuv = ttt.getCuv(du, dv)
Cuv[:, :, 0, 1] *= 0 # set Cuv to 0
Cuv[:, :, 1, 0] *= 0 # set Cvu to 0
n1, n2 = Cuv.shape[0], Cuv.shape[1]

Ks = np.swapaxes(Cuv, 1, 2).reshape(2*n1, 2*n2)

fbar_s = GPRutils.unflat(np.dot(Ks.T, alpha))
# This fbar_s will be the same as dataC.fbar_s